# Importing drivers

In [1]:
from labdrivers.ni import bnc2110
from labdrivers.keithley import keithley2400
from labdrivers.srs import sr830

# Object instantiation

In [2]:
daq = bnc2110(device='Dev1')
keithley = keithley2400(GPIBaddr=22)
lockin = sr830(GPIBaddr=8)

# Importing other useful libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Setting up important constants

In [8]:
quantum_conductance = 0.000077480917310 # in Siemens
current_preamp_sensitivity = 10 ** -5
ac_amplitude = 0.050

# Setting up the SR830 lock-in amplifier

In [9]:
lockin.setFrequency(freq=137.5)
lockin.setInput(i=0)
lockin.setAmplitude(level=ac_amplitude)
lockin.setTimeConst(const_index=9)
lockin.setSensitivity(sens_index=25)

# Making a 2D sweep

As a very simple example, this is just a sweep on a 20 kOhm resistor at room temperature.

In [21]:
columns = ['gate_voltage','bias_voltage','dConductance']
data = pd.DataFrame(columns=columns)

for gate_voltage in range(0,1025,25):
    
    gate = gate_voltage / 100
    keithley.setSourceDC(source='voltage', value=gate)
    
    for bias_voltage in range(-10,11,1):
        
        bias = bias_voltage / 10
        daq.setVoltageOutput(channel='ao1', output=bias)
        
        dConductance = lockin.getSinglePoint(parameter=1)[0] * current_preamp_sensitivity / \
                        ac_amplitude
        
        new_record = pd.DataFrame(np.array([[gate,bias,dConductance]]),
                                 columns=columns)
        data = data.append(new_record)
        
        

In [22]:
data_pivoted = data.pivot(index='gate_voltage',
                         columns='bias_voltage',
                         values='dConductance')

In [23]:
data_pivoted.head()

bias_voltage,-1.0,-0.9,-0.8,-0.7,-0.6,-0.5,-0.4,-0.3,-0.2,-0.1,...,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
gate_voltage,,,,,,,,,,,,,,,,,,,,,
0.00,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,...,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036
0.25,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,...,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036
0.50,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,...,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036
0.75,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,...,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036
1.00,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,...,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036,0.000036


The number here is a bit off from what you are supposed to expect for a 20 kOhm resistor, but otherwise this does exactly what we expect in terms of outputting data.